In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from common.bio.amino_acid import *
from common.bio.smiles import *
from common.preprocessing.dataframe import *
import json
import os

C:\Users\ASUS\Anaconda3\envs\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\Anaconda3\envs\python36\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\ASUS\Anaconda3\envs\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import csv

with open('../data/bmdh_seq_uniprot_full.fasta') as f:
    stripped = (line.split("\n")[0].strip() for line in f)
    lines = (line.split(" ") for line in stripped)
    with open("../data/bmdh_seq_uniprot_full.csv", 'w', newline='') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('Sequence', 'EC class'))
        writer.writerows(lines)

In [3]:
SMILES_COLUMNS=['smiles_product_1','smiles_product_2','smiles_substrate_1','smiles_substrate_2']

In [4]:
MIN_NUM_EXAMPLES = 200

LABEL_COLUMN = "EC number"
VERSION = "Single"
MAX_SEQUENCE_LENGTH=512
IS_VALIDATION_REQUIRED = False
ADD_SMILES=False
NUM_THREADS = 1
DATA_SOURCE = "../data/bmdh_seq_uniprot_full.csv"

## Loading data

In [5]:
original_data = pd.read_csv(DATA_SOURCE, sep=',', skipinitialspace=True, error_bad_lines=False).rename(columns={"EC class": "EC number", 
                                                                                          "Sequence" :"sequence"})
original_data.shape

b'Skipping line 4: expected 2 fields, saw 4\nSkipping line 7809: expected 2 fields, saw 3\nSkipping line 8936: expected 2 fields, saw 3\nSkipping line 9024: expected 2 fields, saw 3\nSkipping line 16289: expected 2 fields, saw 3\nSkipping line 22233: expected 2 fields, saw 3\nSkipping line 22341: expected 2 fields, saw 4\nSkipping line 22696: expected 2 fields, saw 3\nSkipping line 25600: expected 2 fields, saw 3\nSkipping line 27759: expected 2 fields, saw 3\nSkipping line 27763: expected 2 fields, saw 3\nSkipping line 29570: expected 2 fields, saw 4\nSkipping line 33041: expected 2 fields, saw 3\nSkipping line 36722: expected 2 fields, saw 3\nSkipping line 40320: expected 2 fields, saw 3\nSkipping line 40323: expected 2 fields, saw 3\nSkipping line 40368: expected 2 fields, saw 4\nSkipping line 40392: expected 2 fields, saw 3\nSkipping line 40396: expected 2 fields, saw 3\nSkipping line 40653: expected 2 fields, saw 3\nSkipping line 40711: expected 2 fields, saw 4\nSkipping line 4071

(43708, 2)

In [6]:
data = original_data

In [7]:
data.head()

,sequence,EC number
0,MATATSASLFSTVSSSYSKASSIPHSRLQSVKFNSVPSFTGLKSTS...,1.1.1.37
1,MITIIGSGRVGTAAAVIMGLLKIDTKILLIDIIKGLPQGEALDMNH...,1.1.1.37
2,MAKEPMRVLVTGAAGQIGYALVPMIARGVMLGADQPVILHMLDIPP...,1.1.1.37
3,MSEPIRVLVTGAAGQIAYSLLYSIGNGSVFGKDQPIILVLLDITPM...,1.1.1.37
4,MTRTPVNVTVTGAAGQIGYALLFRIASGQLLGADVPVKLRLLEITP...,1.1.1.37


## Adding smiles components

In [8]:
def filter_character(selected_data, character):
    selected_data = selected_data[~selected_data.smiles_product_1.str.contains(character)]
    selected_data = selected_data[~selected_data.smiles_product_2.str.contains(character)]
    selected_data = selected_data[~selected_data.smiles_substrate_1.str.contains(character)]
    selected_data = selected_data[~selected_data.smiles_substrate_2.str.contains(character)]
    return selected_data

In [9]:
def filter_smiles_characters(selected_data):
    selected_data = filter_character(selected_data, "s")
    selected_data = filter_character(selected_data, "F")
    selected_data = filter_character(selected_data, "R")
    selected_data = filter_character(selected_data, "X")
    selected_data = filter_character(selected_data, "6")
    selected_data = filter_character(selected_data, "#")
    selected_data = filter_character(selected_data, "l")
    selected_data = filter_character(selected_data, "@")
    selected_data.smiles_product_1 = selected_data.smiles_product_1.str.replace('R', '*')
    selected_data.smiles_product_2 = selected_data.smiles_product_2.str.replace('R', '*')
    selected_data.smiles_substrate_1 = selected_data.smiles_substrate_1.str.replace('R', '*')
    selected_data.smiles_substrate_2 = selected_data.smiles_substrate_2.str.replace('R', '*')
    return selected_data

In [10]:
# if ADD_SMILES:
#     enzyme_smiles_map = pd.read_csv("../../data/protein/cgan/enzyme_with_smiles.csv", sep='\t', skipinitialspace=True)
#     enzyme_smiles_map = enzyme_smiles_map.fillna("")
#     enzyme_smiles_map = filter_smiles_characters(enzyme_smiles_map)
#     for col in SMILES_COLUMNS:
#         enzyme_smiles_map = enzyme_smiles_map[enzyme_smiles_map[col].str.len() <= 128]
#     merged = pd.merge(data, enzyme_smiles_map, left_on=LABEL_COLUMN, right_on='EC class', how="inner")
#     merged = merged.drop('EC class', axis=1)
#     data = merged

In [11]:
data.head()

,sequence,EC number
0,MATATSASLFSTVSSSYSKASSIPHSRLQSVKFNSVPSFTGLKSTS...,1.1.1.37
1,MITIIGSGRVGTAAAVIMGLLKIDTKILLIDIIKGLPQGEALDMNH...,1.1.1.37
2,MAKEPMRVLVTGAAGQIGYALVPMIARGVMLGADQPVILHMLDIPP...,1.1.1.37
3,MSEPIRVLVTGAAGQIAYSLLYSIGNGSVFGKDQPIILVLLDITPM...,1.1.1.37
4,MTRTPVNVTVTGAAGQIGYALLFRIASGQLLGADVPVKLRLLEITP...,1.1.1.37


## Selecting data

In [12]:
is_test=False
if is_test:
    prefix = "test"
else:
    prefix = "train"

In [13]:
data

,sequence,EC number
0,MATATSASLFSTVSSSYSKASSIPHSRLQSVKFNSVPSFTGLKSTS...,1.1.1.37
1,MITIIGSGRVGTAAAVIMGLLKIDTKILLIDIIKGLPQGEALDMNH...,1.1.1.37
2,MAKEPMRVLVTGAAGQIGYALVPMIARGVMLGADQPVILHMLDIPP...,1.1.1.37
3,MSEPIRVLVTGAAGQIAYSLLYSIGNGSVFGKDQPIILVLLDITPM...,1.1.1.37
4,MTRTPVNVTVTGAAGQIGYALLFRIASGQLLGADVPVKLRLLEITP...,1.1.1.37
...,...,...
43703,MELNLSVKDITGFVLGGHGDDMVPLIRYSYAGGIPLESLIPKHRLE...,1.1.1.37
43704,MYDKNKLFGVTTLDIIRSNTFVAELKGKSATEVEVPVIGGHSGVTI...,1.1.1.37
43705,MKVAVLGAAGGIGQALALLLKTQLPSGSELSLYDIAPVTPGVAVDL...,1.1.1.37
43706,MTATPEVAVSSNTQTLDFDALVLLLEQIFQRHGTSADVARTLALNC...,1.1.1.37


In [14]:
properties = {}
selected_data = data[(data.sequence.str.len() <= MAX_SEQUENCE_LENGTH) &(data.sequence.str.len() >= 64)]
if ADD_SMILES:
    selected_data = filter_smiles_characters(selected_data)
    selected_data =  selected_data[(selected_data[LABEL_COLUMN].str.startswith("3.")) | (selected_data[LABEL_COLUMN].str.startswith("4.")) | (selected_data[LABEL_COLUMN].str.startswith("6."))]
if is_test:
    selected_data = selected_data.groupby(LABEL_COLUMN, group_keys=False).filter(lambda x: len(x) < MIN_NUM_EXAMPLES)
    selected_data = selected_data.groupby(LABEL_COLUMN, group_keys=False).filter(lambda x: len(x) >= 1)
else:
    selected_data = selected_data.groupby(LABEL_COLUMN, group_keys=False).filter(lambda x: len(x) >= MIN_NUM_EXAMPLES)

In [15]:
selected_data = selected_data[selected_data["EC number"] == "1.1.1.37"]

In [16]:
len(selected_data)

43503

In [17]:
selected_data.groupby(LABEL_COLUMN).count()

,sequence
EC number,
1.1.1.37,43503


## Extra preprocessing steps

### Adding label

Converting unique label to int

In [18]:
id_to_category, category_to_id = get_category_to_id_mapping(selected_data, LABEL_COLUMN)
selected_data["Label"] = [ category_to_id[val] for index, val in selected_data[LABEL_COLUMN].iteritems()]
selected_data.head()

There are 1 unique categories


,sequence,EC number,Label
0,MATATSASLFSTVSSSYSKASSIPHSRLQSVKFNSVPSFTGLKSTS...,1.1.1.37,0
1,MITIIGSGRVGTAAAVIMGLLKIDTKILLIDIIKGLPQGEALDMNH...,1.1.1.37,0
2,MAKEPMRVLVTGAAGQIGYALVPMIARGVMLGADQPVILHMLDIPP...,1.1.1.37,0
3,MSEPIRVLVTGAAGQIAYSLLYSIGNGSVFGKDQPIILVLLDITPM...,1.1.1.37,0
4,MTRTPVNVTVTGAAGQIGYALLFRIASGQLLGADVPVKLRLLEITP...,1.1.1.37,0


### Train/test split

In [19]:
from sklearn.model_selection import train_test_split
if IS_VALIDATION_REQUIRED:
    train_data, val_data = train_test_split(selected_data, test_size=0.2)
else:
    train_data = selected_data
    val_data = pd.DataFrame(columns=[selected_data.columns])

In [20]:
train_data.shape, val_data.shape

((43503, 3), (0, 3))

### CSV

In [21]:
if ADD_SMILES:
    PATH = "../data/protein/Length_{}_Examples_{}_{}/".format(MAX_SEQUENCE_LENGTH, MIN_NUM_EXAMPLES, VERSION)
else:
    PATH = "../data/protein/Length_{}_{}/".format(MAX_SEQUENCE_LENGTH, VERSION)

In [22]:
os.makedirs(PATH, exist_ok=True)

In [23]:
def save_as_csv(is_test, data, val_prefix=""):
    # Sequence csv
    seq_data = data[["Label", LABEL_COLUMN, "sequence"]]
    seq_path = os.path.join(PATH, "{}{}_sequences.csv".format(prefix,val_prefix))
    seq_data.to_csv(seq_path, sep='\t', index=None)
    # Reaction csv
    reaction_data = data[["Label", LABEL_COLUMN]].drop_duplicates().sort_values("EC number")
    reaction_path = os.path.join(PATH,"{}{}_reactions.csv".format(prefix, val_prefix))
    reaction_data.to_csv(reaction_path, sep='\t', index=None)

In [24]:
save_as_csv(is_test, train_data)
if IS_VALIDATION_REQUIRED:
    save_as_csv(is_test, val_data, "val_")

### Sequences

In [25]:
def save_sequences_as_tfrecords(original_data, prefix, num_threads = NUM_THREADS):
    data = filter_non_standard_amino_acids(original_data, "sequence")
    data["sequence"] = from_amino_acid_to_id(data, "sequence")
    data = data[["Label", "sequence"]]
    save_as_tfrecords_multithreaded(os.path.join(PATH, prefix), data) 

In [26]:
save_sequences_as_tfrecords(train_data, prefix)
#if IS_VALIDATION_REQUIRED:
#    save_sequences_as_tfrecords(val_data, "val")

Data was stored in ../data/protein/Length_512_Single/train\0.tfrecords
Completed all threads in 9.091354846954346 seconds


### Reactions

In [27]:
def save_reaction_as_tfrecords(original_data, prefix):
    data = original_data[["Label", *SMILES_COLUMNS]].drop_duplicates().sort_values("Label")
    for smiles_col in SMILES_COLUMNS:
        new_col = []
        for index, row in data[smiles_col].iteritems():
            new_col.append([SMILES_CHARACTER_TO_ID[char] for char in row ])
        data[smiles_col] = new_col
    path = os.path.join(PATH, "{}.npy".format(prefix))
    np.save(path, data)
    #save_as_numpy_array(path, data, columns=SMILES_COLUMNS) 

In [28]:
save_reaction_as_tfrecords(train_data, prefix + "_reactions")
# if IS_VALIDATION_REQUIRED:
#     save_reaction_as_tfrecords(val_data, "val_reactions")    

KeyError: "['smiles_substrate_2', 'smiles_product_1', 'smiles_substrate_1', 'smiles_product_2'] not in index"

### Properties

In [29]:
def get_class_mapping(selected_data):
    unique_classes = selected_data[[LABEL_COLUMN, "Label"]].sort_values(LABEL_COLUMN).drop_duplicates()
    mapping = pd.Series(unique_classes[LABEL_COLUMN].values,index=unique_classes.Label).to_dict()
    return mapping

In [30]:
def get_class_counts(selected_data):
    class_counts = selected_data[[LABEL_COLUMN, "Label"]].sort_values(LABEL_COLUMN).groupby(LABEL_COLUMN).count().reset_index()
    counts = pd.Series(class_counts.Label.values,index=class_counts[LABEL_COLUMN]).to_dict()
    return counts

In [31]:
def get_properties(data, validation_required=False):
    properties["seq_length"] = MAX_SEQUENCE_LENGTH
    properties["min_num_examples"] = MIN_NUM_EXAMPLES
    properties["total_records"] = len(selected_data)
    properties["num_of_classes"] = len(data.Label.unique())
    if not validation_required:
        properties["num_examples"] = get_class_counts(data)
        properties["class_mapping"] = get_class_mapping(data)
    return properties

In [32]:
properties = get_properties(selected_data)
properties

{'seq_length': 512,
 'min_num_examples': 200,
 'total_records': 43503,
 'num_of_classes': 1,
 'num_examples': {'1.1.1.37': 43503},
 'class_mapping': {0: '1.1.1.37'}}

In [33]:
if is_test:
    filename = 'properties_test.json'
else:
    filename = 'properties.json'
full_path = os.path.join(PATH, filename)
with open(full_path, 'w') as outfile:
    json.dump(properties, outfile)
    
if IS_VALIDATION_REQUIRED:
    full_path = os.path.join(PATH, 'properties_test.json')
    with open(full_path, 'w') as outfile:
        json.dump(get_properties(test, True), outfile)
        print("Test Properties saved in {}".format(full_path))
    
print("Properties saved in {}".format(full_path))

Properties saved in ../data/protein/Length_512_Single/properties.json


 #### !!!!!!!!!!!!!!! That is all done !!!!!!!!!!!!!

# End of preprocessing